In [1]:
import caffe
import numpy as np

In [2]:
r = 240

In [3]:
net = caffe.Net("./models/no_bn/TestModel.prototxt", "./models/no_bn/TestModel.caffemodel", caffe.TEST)

In [4]:
weight, bias = net.params['fc5_']

In [5]:
U, sigma, VT = np.linalg.svd(weight.data, full_matrices=False)

In [6]:
sigma[:r].sum()/sigma.sum()

0.90778905

In [7]:
net2 = caffe.Net("./models/fc_svd/TestModel.prototxt", caffe.TEST)

In [8]:
for key in net2.params.keys():
    if key == 'fc5_1':
        net2.params[key][0].data[...] = np.dot(np.eye(r) * sigma[:r], VT[:r])
    elif key =='fc5_':
        net2.params[key][0].data[...] = U[:,:r]
        net2.params[key][1].data[...] = bias.data
    else:
        net2.params[key][0].data[...] = net.params[key][0].data
        net2.params[key][1].data[...] = net.params[key][1].data

In [9]:
net2.save("./models/fc_svd/TestModel.caffemodel")